In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from tqdm.keras import TqdmCallback
import plotly.graph_objects as go
import numpy as np
import tensorflow as tf
from tqdm import tqdm

In [2]:
def build_model_regression(lr_=0.05,  num_input_data=7, last_activation='relu'):
    model = models.Sequential()
    model.add(layers.Dense(32, activation='relu', input_shape=(num_input_data,)))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1,  activation=last_activation)) # sigmoid
    model.compile(optimizer = optimizers.RMSprop(learning_rate= lr_),
                loss = 'mse',
                metrics = 'mse')
    return model

def preparar_datos(time_series, window = 5):
    time = time_series[time_series.columns[0]].values
    values = time_series[time_series.columns[1]].values
    X_list = []
    Y_list = []
    for i in range(len(time)-window):
        X_i = values[i:i+window]
        mean = np.array(X_i).mean()
        std = np.array(X_i).std()
        X_i = np.append(X_i, std)
        X_i = np.append(X_i, mean)
        X_list.append(X_i)
        Y_list.append(values[i+window])
    X = np.vstack(X_list)
    Y = np.vstack(Y_list)
    return X, Y

In [3]:
credentials = service_account.Credentials.from_service_account_file('tidy-hold-359719-964ba4f0d1d8.json') #credenciales
project_id = 'tidy-hold-359719' #nombre del proyecto
client = bigquery.Client(credentials= credentials,project=project_id)
query="""SELECT * FROM `tidy-hold-359719.comision_latinoamericana.temperatura`"""
temperatura = client.query(query).to_dataframe()
query="""SELECT * FROM `tidy-hold-359719.comision_latinoamericana.datos_ONU`"""
datos_ONU = client.query(query).to_dataframe()
query="""SELECT * FROM `tidy-hold-359719.comision_latinoamericana.energyco2`"""
energyco2 = client.query(query).to_dataframe()

In [4]:
ts1 = temperatura.groupby('Anio').mean().reset_index()

In [5]:
tabla = energyco2[(energyco2.Pais != 'Estados Unidos') & (energyco2.Pais != 'Canadá')]
ts2 = tabla.drop(tabla[tabla['Emisiones_de_CO2'] == -1].index).groupby('Anio').sum().reset_index()

In [6]:
datos_ONU = datos_ONU[(datos_ONU.Pais != 'Estados Unidos') & (datos_ONU.Pais != 'Canadá')]

In [7]:

tabla = datos_ONU[['Anio','intensidad_energetica_medida_en_terminos_de_energia_primaria_y_PBI']]
ts3 = tabla.drop(tabla[tabla['intensidad_energetica_medida_en_terminos_de_energia_primaria_y_PBI']==-1].index).groupby('Anio').mean().reset_index().rename(columns={'intensidad_energetica_medida_en_terminos_de_energia_primaria_y_PBI':'intensidad'})

In [8]:
datos_ONU['prom'] = (datos_ONU['proporcion_de_la_poblacion_con_acceso_a_elecricidad'] + datos_ONU['proporcion_de_la_poblacion_con_dependencia_primaria_a_energias_limpias'])/2
ts4 = datos_ONU[['Anio','prom']].groupby('Anio').mean().reset_index()

In [9]:
ts5 = datos_ONU[['Anio', 'proporcion_de_energias_renovables_del_total_consumido']].groupby('Anio').mean().reset_index().rename(columns={'proporcion_de_energias_renovables_del_total_consumido':'prop_ren'})

In [10]:
# Preparamos datos
X_1, Y_1 = preparar_datos(ts1)
X_2, Y_2 = preparar_datos(ts2)
X_3, Y_3 = preparar_datos(ts3)
X_4, Y_4 = preparar_datos(ts4)
X_5, Y_5 = preparar_datos(ts5)

# Arreglamos el tipo de dato de X_4
X_4 = np.array(X_4, dtype=float)


----
## Ts1

In [11]:
# Entrenamos modelo
intentos = 30
min_calif = 9999999
for i in tqdm(range(intentos), desc='¡Estamos trabajando en encontrar un mejor modelo!: '):
    x_train,x_test,y_train,y_test=train_test_split(X_1,Y_1,test_size=0.25)
    x_train = tf.convert_to_tensor(x_train)
    x_test = tf.convert_to_tensor(x_test)
    y_train = tf.convert_to_tensor(y_train)
    y_test = tf.convert_to_tensor(y_test)

    model_=build_model_regression(0.001)
    hist = model_.fit(x_train, y_train,
            batch_size=100,
            epochs= 50,
            verbose = 0)
    Y_1_p = model_.predict(tf.convert_to_tensor(X_1), verbose = 0)
    calif = mean_squared_error(Y_1, Y_1_p, squared=False)
    if min_calif > calif:
        min_calif = calif
        model_1 = model_
        x_train_1 = x_train
        y_train_1 = y_train
        x_test_1 = x_test
        y_test_1 = y_test
model_1.save('model_1.h5')
print('Hemos encontrado el mejor modelo, lo puedes encontrar como "model_1.h5"')

¡Estamos trabajando en encontrar un mejor modelo!: 100%|██████████| 30/30 [00:31<00:00,  1.06s/it]

Hemos encontrado el mejor modelo, lo puedes encontrar como "model_1.h5"


In [12]:
# Graficamos Resultados
model = models.load_model('model_1.h5')

Y_1_p = model.predict(tf.convert_to_tensor(X_1), verbose = 0)
rmse_1 = mean_squared_error(Y_1, Y_1_p, squared=False)
mse_1 = mean_squared_error(Y_1, Y_1_p)
mae_1 = mean_absolute_error(Y_1, Y_1_p)
r2_1 = r2_score(Y_1, Y_1_p)
y_train_1_p = model.predict(tf.convert_to_tensor(x_train_1), verbose = 0)
rmse_1_tr = mean_squared_error(y_train_1, y_train_1_p, squared=False)
mse_1_tr = mean_squared_error(y_train_1, y_train_1_p)
mae_1_tr = mean_absolute_error(y_train_1, y_train_1_p)
r2_1_tr = r2_score(y_train_1, y_train_1_p)
y_test_1_p = model.predict(tf.convert_to_tensor(x_test_1), verbose = 0)
rmse_1_te = mean_squared_error(y_test_1, y_test_1_p, squared=False)
mse_1_te = mean_squared_error(y_test_1, y_test_1_p)
mae_1_te = mean_absolute_error(y_test_1, y_test_1_p)
r2_1_te = r2_score(y_test_1, y_test_1_p)

print('RMSE:',rmse_1, rmse_1_tr, rmse_1_te)
print('MSE:',mse_1, mse_1_tr, mse_1_te)
print('MAE:',mae_1, mae_1_tr, mae_1_te)
print('R2:',r2_1, r2_1_tr, r2_1_te)


fig = go.Figure()
fig.add_trace(go.Scatter(y = Y_1_p.reshape(-1)+2*rmse_1, mode = 'lines',line_color='rgba(0,0,0,0)',showlegend=False))
fig.add_trace(go.Scatter(y = Y_1_p.reshape(-1)-2*rmse_1, fill='tonexty', mode = 'lines',line_color='rgba(100,100,255,1)', line_width=0,showlegend=False))
fig.add_trace(go.Scatter(y = Y_1.reshape(-1), name='Real', mode = 'markers', line_color='black'))
fig.add_trace(go.Scatter(y = Y_1_p.reshape(-1), name='Model', line_color='rgb(50,50,255)'))
fig.show()

RMSE: 0.1698390777406583 0.17009920032774825 0.1690563085329002
MSE: 0.02884531232779737 0.028933737952139434 0.028580035454771145
MAE: 0.13286085918031906 0.13294696764014238 0.13260253380084908
R2: 0.7818627618680417 0.7807835450475606 0.7757153738882653


----
## Ts2

In [13]:
# Entrenamos modelo
intentos = 30
min_calif = 9999999
for i in tqdm(range(intentos), desc='¡Estamos trabajando en encontrar un mejor modelo!: '):
    x_train,x_test,y_train,y_test=train_test_split(X_2,Y_2,test_size=0.25)
    x_train = tf.convert_to_tensor(x_train)
    x_test = tf.convert_to_tensor(x_test)
    y_train = tf.convert_to_tensor(y_train)
    y_test = tf.convert_to_tensor(y_test)

    model_=build_model_regression(0.01)
    hist = model_.fit(x_train, y_train,
            batch_size=100,
            epochs= 50,
            verbose = 0)
    Y_2_p = model_.predict(tf.convert_to_tensor(X_2), verbose = 0)
    calif = mean_squared_error(Y_2, Y_2_p, squared=False)
    if min_calif > calif:
        min_calif = calif
        model_2 = model_
        hist_3 = hist
        x_train_2 = x_train
        y_train_2 = y_train
        x_test_2 = x_test
        y_test_2 = y_test
        
model_2.save('model_2.h5')
print('Hemos encontrado el mejor modelo, lo puedes encontrar como "model_2.h5"')

¡Estamos trabajando en encontrar un mejor modelo!:  10%|█         | 3/30 [00:03<00:31,  1.17s/it]

¡Estamos trabajando en encontrar un mejor modelo!:  13%|█▎        | 4/30 [00:04<00:29,  1.12s/it]

¡Estamos trabajando en encontrar un mejor modelo!: 100%|██████████| 30/30 [00:32<00:00,  1.09s/it]

Hemos encontrado el mejor modelo, lo puedes encontrar como "model_2.h5"


In [14]:
# Graficamos Resultados
model = models.load_model('model_2.h5')
Y_2_p = model.predict(tf.convert_to_tensor(X_2), verbose = 0)
rmse_2 = mean_squared_error(Y_2, Y_2_p, squared=False)
mse_2 = mean_squared_error(Y_2, Y_2_p)
mae_2 = mean_absolute_error(Y_2, Y_2_p)
r2_2 = r2_score(Y_2, Y_2_p)
y_train_2_p = model.predict(tf.convert_to_tensor(x_train_2), verbose = 0)
rmse_2_tr = mean_squared_error(y_train_2, y_train_2_p, squared=False)
mse_2_tr = mean_squared_error(y_train_2, y_train_2_p)
mae_2_tr = mean_absolute_error(y_train_2, y_train_2_p)
r2_2_tr = r2_score(y_train_2, y_train_2_p)
y_test_2_p = model.predict(tf.convert_to_tensor(x_test_2), verbose = 0)
rmse_2_te = mean_squared_error(y_test_2, y_test_2_p, squared=False)
mse_2_te = mean_squared_error(y_test_2, y_test_2_p)
mae_2_te = mean_absolute_error(y_test_2, y_test_2_p)
r2_2_te = r2_score(y_test_2, y_test_2_p)

print('RMSE:',rmse_2, rmse_2_tr, rmse_2_te)
print('MSE:',mse_2, mse_2_tr, mse_2_te)
print('MAE:',mae_2, mae_2_tr, mae_2_te)
print('R2:',r2_2, r2_2_tr, r2_2_te)


fig = go.Figure()
fig.add_trace(go.Scatter(y = Y_2_p.reshape(-1)+2*rmse_2, mode = 'lines',line_color='rgba(0,0,0,0)',showlegend=False))
fig.add_trace(go.Scatter(y = Y_2_p.reshape(-1)-2*rmse_2, fill='tonexty', mode = 'lines',line_color='rgba(100,100,255,1)', line_width=0,showlegend=False))
fig.add_trace(go.Scatter(y = Y_2.reshape(-1), name='Real', mode = 'markers', line_color='black'))
fig.add_trace(go.Scatter(y = Y_2_p.reshape(-1), name='Model', line_color='rgb(50,50,255)'))
fig.show()

RMSE: 53.534496501134264 58.3353086145968 37.06875436628575
MSE: 2865.9423156299567 3403.0082311602514 1374.0925502680288
MAE: 40.528207341381446 43.49230228531252 32.29461027490624
R2: 0.969303618356432 0.9578656017946376 0.9876790471590952


----
## Ts3

In [15]:
# Entrenamos modelo
intentos = 30
min_calif = 9999999
for i in tqdm(range(intentos), desc='¡Estamos trabajando en encontrar un mejor modelo!: '):
    x_train,x_test,y_train,y_test=train_test_split(X_3,Y_3,test_size=0.25)
    x_train = tf.convert_to_tensor(x_train)
    x_test = tf.convert_to_tensor(x_test)
    y_train = tf.convert_to_tensor(y_train)
    y_test = tf.convert_to_tensor(y_test)

    model_=build_model_regression(0.01)
    hist = model_.fit(x_train, y_train,
            batch_size=100,
            epochs= 50,
            verbose = 0)
    Y_3_p = model_.predict(tf.convert_to_tensor(X_3), verbose = 0)
    calif = mean_squared_error(Y_3, Y_3_p, squared=False)
    if min_calif > calif:
        min_calif = calif
        hist_3 = hist
        model_3 = model_
        x_train_3 = x_train
        y_train_3 = y_train
        x_test_3 = x_test
        y_test_3 = y_test
model_3.save('model_3.h5')
print('Hemos encontrado el mejor modelo, lo puedes encontrar como "model_3.h5"')

¡Estamos trabajando en encontrar un mejor modelo!: 100%|██████████| 30/30 [00:32<00:00,  1.10s/it]

Hemos encontrado el mejor modelo, lo puedes encontrar como "model_3.h5"


In [16]:
# Graficamos Resultados
model = models.load_model('model_3.h5')
Y_3_p = model.predict(tf.convert_to_tensor(X_3), verbose = 0)
rmse_3 = mean_squared_error(Y_3, Y_3_p, squared=False)
mse_3 = mean_squared_error(Y_3, Y_3_p)
mae_3 = mean_absolute_error(Y_3, Y_3_p)
r2_3 = r2_score(Y_3, Y_3_p)
y_train_3_p = model.predict(tf.convert_to_tensor(x_train_3), verbose = 0)
rmse_3_tr = mean_squared_error(y_train_3, y_train_3_p, squared=False)
mse_3_tr = mean_squared_error(y_train_3, y_train_3_p)
mae_3_tr = mean_absolute_error(y_train_3, y_train_3_p)
r2_3_tr = r2_score(y_train_3, y_train_3_p)
y_test_3_p = model.predict(tf.convert_to_tensor(x_test_3), verbose = 0)
rmse_3_te = mean_squared_error(y_test_3, y_test_3_p, squared=False)
mse_3_te = mean_squared_error(y_test_3, y_test_3_p)
mae_3_te = mean_absolute_error(y_test_3, y_test_3_p)
r2_3_te = r2_score(y_test_3, y_test_3_p)

print('RMSE:',rmse_3, rmse_3_tr, rmse_3_te)
print('MSE:',mse_3, mse_3_tr, mse_3_te)
print('MAE:',mae_3, mae_3_tr, mae_3_te)
print('R2:',r2_3, r2_3_tr, r2_3_te)


fig = go.Figure()
fig.add_trace(go.Scatter(y = Y_3_p.reshape(-1)+2*rmse_3, mode = 'lines',line_color='rgba(0,0,0,0)',showlegend=False))
fig.add_trace(go.Scatter(y = Y_3_p.reshape(-1)-2*rmse_3, fill='tonexty', mode = 'lines',line_color='rgba(100,100,255,1)', line_width=0,showlegend=False))
fig.add_trace(go.Scatter(y = Y_3.reshape(-1), name='Real', mode = 'markers', line_color='black'))
fig.add_trace(go.Scatter(y = Y_3_p.reshape(-1), name='Model', line_color='rgb(50,50,255)'))
fig.show()

RMSE: 0.25312680226811324 0.24431649149364038 0.2759083372353149
MSE: 0.0640731780264805 0.059690548015762054 0.07612541055595626
MAE: 0.24444159410264765 0.23801775464144628 0.2621071526209514
R2: -4.353453143307906 -2.838595516324198 -40.7451174624713


----
## Ts4

In [17]:
# Entrenamos modelo
intentos = 30
min_calif = 9999999
for i in tqdm(range(intentos), desc='¡Estamos trabajando en encontrar un mejor modelo!: '):
    x_train,x_test,y_train,y_test=train_test_split(X_4,Y_4,test_size=0.25)
    x_train = tf.convert_to_tensor(x_train)
    x_test = tf.convert_to_tensor(x_test)
    y_train = tf.convert_to_tensor(y_train)
    y_test = tf.convert_to_tensor(y_test)

    model_=build_model_regression(lr_=0.001)
    hist = model_.fit(x_train, y_train,
            batch_size=100,
            epochs= 50,
            verbose = 0)
    Y_4_p = model_.predict(tf.convert_to_tensor(X_4), verbose = 0)
    calif = mean_squared_error(Y_4, Y_4_p, squared=False)
    if min_calif > calif:
        min_calif = calif
        hist_4 = hist
        model_4 = model_
        x_train_4 = x_train
        y_train_4 = y_train
        x_test_4 = x_test
        y_test_4 = y_test
model_4.save('model_4.h5')
print('Hemos encontrado el mejor modelo, lo puedes encontrar como "model_4.h5"')

¡Estamos trabajando en encontrar un mejor modelo!: 100%|██████████| 30/30 [00:31<00:00,  1.05s/it]

Hemos encontrado el mejor modelo, lo puedes encontrar como "model_4.h5"


In [18]:
# Graficamos Resultados
model = models.load_model('model_4.h5')
Y_4_p = model.predict(tf.convert_to_tensor(X_4), verbose = 0)
rmse_4 = mean_squared_error(Y_4, Y_4_p, squared=False)
mse_4 = mean_squared_error(Y_4, Y_4_p)
mae_4 = mean_absolute_error(Y_4, Y_4_p)
r2_4 = r2_score(Y_4, Y_4_p)
y_train_4_p = model.predict(tf.convert_to_tensor(x_train_4), verbose = 0)
rmse_4_tr = mean_squared_error(y_train_4, y_train_4_p, squared=False)
mse_4_tr = mean_squared_error(y_train_4, y_train_4_p)
mae_4_tr = mean_absolute_error(y_train_4, y_train_4_p)
r2_4_tr = r2_score(y_train_4, y_train_4_p)
y_test_4_p = model.predict(tf.convert_to_tensor(x_test_4), verbose = 0)
rmse_4_te = mean_squared_error(y_test_4, y_test_4_p, squared=False)
mse_4_te = mean_squared_error(y_test_4, y_test_4_p)
mae_4_te = mean_absolute_error(y_test_4, y_test_4_p)
r2_4_te = r2_score(y_test_4, y_test_4_p)

print('RMSE:',rmse_4, rmse_4_tr, rmse_4_te)
print('MSE:',mse_4, mse_4_tr, mse_4_te)
print('MAE:',mae_4, mae_4_tr, mae_4_te)
print('R2:',r2_4, r2_4_tr, r2_4_te)


fig = go.Figure()
fig.add_trace(go.Scatter(y = Y_4_p.reshape(-1)+2*rmse_4, mode = 'lines',line_color='rgba(0,0,0,0)',showlegend=False))
fig.add_trace(go.Scatter(y = Y_4_p.reshape(-1)-2*rmse_4, fill='tonexty', mode = 'lines',line_color='rgba(100,100,255,1)', line_width=0,showlegend=False))
fig.add_trace(go.Scatter(y = Y_4.reshape(-1), name='Real', mode = 'markers', line_color='black'))
fig.add_trace(go.Scatter(y = Y_4_p.reshape(-1), name='Model', line_color='rgb(50,50,255)'))
fig.show()

RMSE: 0.227750120808076 0.2048540415490571 0.2812626891326804
MSE: 0.051870117528093215 0.04196517833898281 0.0791087002981468
MAE: 0.18835649503843116 0.16517357799899007 0.2521095168968941
R2: 0.9893468391716236 0.9914505055627221 0.9796459477944228


----
## Ts5

In [19]:
# Entrenamos modelo
intentos = 30
min_calif = 9999999
for i in tqdm(range(intentos), desc='¡Estamos trabajando en encontrar un mejor modelo!: '):
    x_train,x_test,y_train,y_test=train_test_split(X_5,Y_5,test_size=0.25)
    x_train = tf.convert_to_tensor(x_train)
    x_test = tf.convert_to_tensor(x_test)
    y_train = tf.convert_to_tensor(y_train)
    y_test = tf.convert_to_tensor(y_test)

    model_=build_model_regression(0.001)
    hist = model_.fit(x_train, y_train,
            batch_size=100,
            epochs= 50,
            verbose = 0)
    Y_5_p = model_.predict(tf.convert_to_tensor(X_5), verbose = 0)
    calif = mean_squared_error(Y_5, Y_5_p, squared=False)
    if min_calif > calif:
        min_calif = calif
        hist_5 = hist
        model_5 = model_
        x_train_5 = x_train
        y_train_5 = y_train
        x_test_5 = x_test
        y_test_5 = y_test
model_5.save('model_5.h5')
print('Hemos encontrado el mejor modelo, lo puedes encontrar como "model_5.h5"')

¡Estamos trabajando en encontrar un mejor modelo!: 100%|██████████| 30/30 [00:31<00:00,  1.06s/it]

Hemos encontrado el mejor modelo, lo puedes encontrar como "model_5.h5"


In [20]:
# Graficamos Resultados
model = models.load_model('model_5.h5')
Y_5_p = model.predict(tf.convert_to_tensor(X_5), verbose = 0)
rmse_5 = mean_squared_error(Y_5, Y_5_p, squared=False)
mse_5 = mean_squared_error(Y_5, Y_5_p)
mae_5 = mean_absolute_error(Y_5, Y_5_p)
r2_5 = r2_score(Y_5, Y_5_p)
y_train_5_p = model.predict(tf.convert_to_tensor(x_train_5), verbose = 0)
rmse_5_tr = mean_squared_error(y_train_5, y_train_5_p, squared=False)
mse_5_tr = mean_squared_error(y_train_5, y_train_5_p)
mae_5_tr = mean_absolute_error(y_train_5, y_train_5_p)
r2_5_tr = r2_score(y_train_5, y_train_5_p)
y_test_5_p = model.predict(tf.convert_to_tensor(x_test_5), verbose = 0)
rmse_5_te = mean_squared_error(y_test_5, y_test_5_p, squared=False)
mse_5_te = mean_squared_error(y_test_5, y_test_5_p)
mae_5_te = mean_absolute_error(y_test_5, y_test_5_p)
r2_5_te = r2_score(y_test_5, y_test_5_p)

print('RMSE:',rmse_5, rmse_5_tr, rmse_5_te)
print('MSE:',mse_5, mse_5_tr, mse_5_te)
print('MAE:',mae_5, mae_5_tr, mae_5_te)
print('R2:',r2_5, r2_5_tr, r2_5_te)


fig = go.Figure()
fig.add_trace(go.Scatter(y = Y_5_p.reshape(-1)+2*rmse_5, mode = 'lines',line_color='rgba(0,0,0,0)',showlegend=False))
fig.add_trace(go.Scatter(y = Y_5_p.reshape(-1)-2*rmse_5, fill='tonexty', mode = 'lines',line_color='rgba(100,100,255,1)', line_width=0,showlegend=False))
fig.add_trace(go.Scatter(y = Y_5.reshape(-1), name='Real', mode = 'markers', line_color='black'))
fig.add_trace(go.Scatter(y = Y_5_p.reshape(-1), name='Model', line_color='rgb(50,50,255)'))
fig.show()

RMSE: 0.3940116328224017 0.3497349969301271 0.49578656020906586
MSE: 0.15524516679937508 0.122314568077716 0.24580431328393768
MAE: 0.33394039654475427 0.2966601475242999 0.4364610813510037
R2: 0.8730549081081576 0.8351882091195175 0.5462648276821122


----

In [102]:
model = models.load_model('model_4.h5')
X_f, Y_f = preparar_datos(ts4)
X_f = np.array(X_f, dtype=float)
X_f = tf.convert_to_tensor(X_f)
Y_f_p = model.predict(X_f, verbose=0).reshape(-1)
ts4_c = ts4.copy()
for i in range(11):
    X_, Y_ = preparar_datos(ts4_c)
    X_ = np.array(X_, dtype=float)
    X_ = tf.convert_to_tensor(X_[-1].reshape(1,-1))
    Y_proy = model.predict(X_, verbose = 0)[0][0]
    Anio_pr = ts4_c['Anio'].values[-1] + 1
    ts4_c.loc[len(ts4_c)] = [Anio_pr, Y_proy]
    Y_f_p = np.append(Y_f_p, Y_proy)
anios_p = ts4_c.Anio.values[5:]
anios_proy = np.arange(2020,2031)
proms = Y_f_p[-11:]
max = np.array([])
min = np.array([])
error = 2*rmse_4
suma = 0
for prom in proms:
    suma += error
    max = np.append(max,prom+suma)
    min = np.append(min,prom-suma)
fig = go.Figure()
fig.add_trace(go.Scatter(x = anios_proy, y = max, mode = 'lines',line_color='rgba(0,0,0,0)',showlegend=False))
fig.add_trace(go.Scatter(x = anios_proy, y = min, fill='tonexty', mode = 'lines',line_color='rgba(100,100,255,1)', line_width=0,showlegend=False))
fig.add_trace(go.Scatter(x = anios_p, y = Y_f_p, mode = 'lines', line_color = 'rgb(50,50,255)', name = 'model'))
fig.add_trace(go.Scatter(x = ts4.Anio, y = ts4.prom, mode = 'markers', line_color = 'black', name = 'Real'))
fig.show()


In [133]:
model = models.load_model('model_3.h5')
ts = ts3.copy()
num_proy = 11
rmse = rmse_3

time_col_name = ts.columns[0]
value_col_name = ts.columns[1]
last_anio = ts[time_col_name].values[-1]

X, Y = preparar_datos(ts)
X = np.array(X, dtype=float)
X = tf.convert_to_tensor(X)
Y_pred_proy = model.predict(X, verbose=0).reshape(-1)

ts_c = ts.copy()
for i in range(num_proy):
    X_last, _ = preparar_datos(ts_c)
    X_last = np.array(X_last, dtype=float)
    X_last = tf.convert_to_tensor(X_last[-1].reshape(1,-1))
    Y_proy = model.predict(X_last, verbose = 0)[0][0]
    Anio_next = ts_c[time_col_name].values[-1] + 1
    ts_c.loc[len(ts_c)] = [Anio_next, Y_proy]
    Y_pred_proy = np.append(Y_pred_proy, Y_proy)
anios_pred_proy = ts_c.Anio.values[5:]
Y_proy = Y_pred_proy[-num_proy:]
Y_pred = Y_pred_proy[:-num_proy]
error = 2*rmse
max_arr = Y_pred + error
min_arr = Y_pred - error
suma_error = error
i = 0
for value in Y_proy:
    i += 1 
    max_arr = np.append(max_arr,value+suma_error)
    min_arr = np.append(min_arr,value-suma_error)
    suma_error += error*0.9**i
    suma_error += 0
fig = go.Figure()
fig.add_trace(go.Scatter(x = anios_pred_proy, y = max_arr, mode = 'lines',line_color='rgba(0,0,0,0)',showlegend=False))
fig.add_trace(go.Scatter(x = anios_pred_proy, y = min_arr, fill='tonexty', mode = 'lines',line_color='rgba(100,100,255,1)', line_width=0,showlegend=False))
fig.add_trace(go.Scatter(x = anios_pred_proy, y = Y_pred_proy, mode = 'lines', line_color = 'rgb(50,50,255)', name = 'model'))
fig.add_trace(go.Scatter(x = ts[time_col_name], y = ts[value_col_name], mode = 'markers', line_color = 'black', name = 'Real'))
fig.show()

In [113]:
Y_pred + error

array([23.644115, 23.56026 , 23.502636, 23.526539, 23.496538, 23.405155,
       23.479374, 23.547726, 23.42976 , 23.464005, 23.584192, 23.463638,
       23.444279, 23.491756, 23.532238, 23.43711 , 23.451443, 23.462967,
       23.47036 , 23.483028, 23.477423, 23.666733, 23.54797 , 23.591305,
       23.611187, 23.60713 , 23.614391, 23.712841, 23.643204, 23.643791,
       23.66224 , 23.740019, 23.696917, 23.576036, 23.708246, 23.759361,
       23.835339, 23.778477, 23.760563, 23.884426, 23.800909, 23.763325,
       23.800842, 23.806374, 23.675003, 23.603355, 23.770807, 23.703194,
       23.741436, 23.580902, 23.646797, 23.56332 , 23.772255, 23.811302,
       23.770638, 23.862808, 23.899643, 23.814934, 23.874088, 23.737797,
       23.797401, 23.753723, 23.70174 , 23.630283, 23.906355, 23.678349,
       23.615253, 23.866714, 23.787434, 23.593216, 23.684402, 23.602068,
       23.815807, 23.724863, 23.835785, 24.006199, 23.93286 , 24.03215 ,
       24.145285, 23.87152 , 23.99653 , 23.99621 , 

In [107]:
Y_pred_proy

array([81.386894, 81.804924, 82.575294, 83.15806 , 83.811386, 84.31926 ,
       85.019684, 85.50497 , 86.09501 , 86.58853 , 87.084816, 87.56896 ,
       88.01436 , 88.38065 , 88.763374, 92.24467 , 92.24467 , 92.24467 ,
       92.24467 , 92.24467 , 92.24467 , 92.24467 , 92.24467 , 92.24467 ],
      dtype=float32)